In [1]:
import pandas as pd
import numpy as np 

In [35]:
rf = pd.read_csv("./from_skip_files/Restaurant_item(LowerCase).csv")
rf_c = pd.read_csv('./derived_files/Cuisine_Restaraunt.csv')

In [36]:
print(rf['short_name'].nunique())
print(rf['short_name'].str.contains('DNU').sum())
print(rf.head())

2354
0
             short_name      menuGroups_menuItems_name  menuGroups_name  \
0  1600 world bier haus          Steak Sandwich (6 oz)            Mains   
1  1600 world bier haus  Roasted Butternut Squash Soup  Soup and Salads   
2  1600 world bier haus           Jumbo Prawn Linguine            Pasta   
3  1600 world bier haus              Haus Greens Salad  Soup and Salads   
4  1600 world bier haus                 Kids Ice Cream        Kids Menu   

                    menuGroups_menuItems_description  
0  Six-ounce Alberta flat iron, frizzled onions, ...  
1                              paprika, herbed crema  
2  jumbo prawns, butter garlic, brunoise onion, s...  
3  Vegetarian. Dried cranberries, toasted pumpkin...  
4  Belgium waffle with vanilla bean ice cream, ch...  


In [37]:
rf = rf.fillna("")
print(rf.columns)
rf['item_description'] = rf['menuGroups_menuItems_name']+" "+rf["menuGroups_name"]+" "+rf["menuGroups_menuItems_description"]
rf = rf.drop(["menuGroups_menuItems_name","menuGroups_name","menuGroups_menuItems_description"], axis=1)
print(rf)

Index(['short_name', 'menuGroups_menuItems_name', 'menuGroups_name',
       'menuGroups_menuItems_description'],
      dtype='object')
                  short_name  \
0       1600 world bier haus   
1       1600 world bier haus   
2       1600 world bier haus   
3       1600 world bier haus   
4       1600 world bier haus   
...                      ...   
284432   zyn the wine market   
284433   zyn the wine market   
284434   zyn the wine market   
284435   zyn the wine market   
284436   zyn the wine market   

                                         item_description  
0       Steak Sandwich (6 oz) Mains Six-ounce Alberta ...  
1       Roasted Butternut Squash Soup Soup and Salads ...  
2       Jumbo Prawn Linguine Pasta jumbo prawns, butte...  
3       Haus Greens Salad Soup and Salads Vegetarian. ...  
4       Kids Ice Cream Kids Menu Belgium waffle with v...  
...                                                   ...  
284432  Park Distillery Alpine Gin Spirits (ABV 45%) (...  


In [40]:
intersection = set(rf['short_name'].unique()) & set(rf_c['short_name'].to_list())
diff = set(rf['short_name'].unique()) - set(rf_c['short_name'].to_list())
print(rf_c['short_name'].nunique())
print(rf['short_name'].nunique())
print(len(intersection))
print(len(diff))

1511
2354
1160
1194


In [41]:
ethnic_or_cultural_items = [
    "Convenience",
    "Indian",
    "Italian",
    "Chinese",
    "Vietnamese",
    "Japanese",
    "Mediterranean",
    "Middle Eastern",
    "Mexican",
    "Korean",
    "Thai",
    "French",
    "African",
    "Latin American",
    "Ethiopian",
    "Caribbean",
    "Filipino",
    "Spanish",
    "German",
]

In [42]:
# Map of cuisine type to keywords that fit under that cuisine
cuisine_map = {
    "Chicken" : ["chicken"],
    "Fries" : ["fries"],
    "Beef" : ["beef"],
    "Pork" : ["pork", "bacon", "pepperoni"],
    "Rice" : ["rice"],
    "Lamb" : ["lamb"],
    "Vegetarian" : ["vegan", "vegetarian", "veggie", "beyond meat"],
    "Sandwiches & Subs" : ["sandwich", "sub", "wrap", "blt"],
    "Desserts" : ["blizzard", "ice cream", "frozen", "dessert", "chocolate", "drizzle", "desserts", "milkshake", "candy", 
                  "candies", "sundae", "oreo", "skor", "brownie", "shake", "tiramisu", "timbits", "ben and jerry", 
                  "cheesecake", "cookie"],
    "Canadian" : ["canadian", "canadien", "alberta","poutine"],
    "Fast food" : ["fast food","combo", "meal"],
    "Burgers" : ["burger", "patty", "mcdouble", "big mac", "quarter pounder"],
    "Seafood" : ["fish", "seafood", "shrimp", "crab", "lobster", "prawn", "seaweed", 
                 "salmon", "tuna", "poke", "calamari", "squid", "fish and chips"],
    "Healthy" : ["organic", "health", "protein", "salad", "fresh", "tofu",  "fruit", "water", "vegetable", "smoothie", "parfait"],
    "Pizza" : ["pizza"],
    "Breakfast & Brunch" : ["egg", "toast", "benedict", "breakfast", "brunch", "cereal", "pancake", "waffle", "hash brown"],
    "Coffee/Tea" : ["coffee", "tea", "americano", "cappuccino", "latte", "cafe", "chai", "london fog"],
    "Alcohol" : ["beer", "wine", "liquor", "budweiser", "bud light", "spirits", "corona", "stella artois", 
                 "michelob ultra", "mike's hard", "labatt", "sauvignon", "smirnoff", "vodka", "whisky", "cognac",
                 "white claw", "pinot noir"],
    "Noodles" : ["noodle", "vermicelli"],
    "Pub food" : ["wing", "onion ring", "wedge", "mac & cheese", "mac and cheese", "gravy", "mashed potato", "breadsticks"], 
    "Indian" : ["indian", "naan", "nan", "samosa", "masala", "aloo", "paneer", "biryani", "tandoori", "roti", "tikka"],
    "Italian" : ["italian", "pasta", "spaghetti", "penne", "fettuccini", "lasagna", "lasagne", "linguini", "ravioli", "tortellini", "meatball", "canoli"],
    "Bakery" : ["danish", "cake", "bun", "donut", "muffin","bagel", "doughnut", "pie", "scone", "rolls", "loaf"],
    "Barbecue" : ["barbecue", "bbq", "grill", "buffalo"],
    "Chinese" : ["chinese", "china", "hot pot", "wonton", "cantonese", "mein", "gyoza"],
    "Vietnamese" : ["vietnamese", "pho", "viet", "bun cha","ca kho to"],
    "Japanese" : ["japanese","japan", "ramen", "sashimi", "teriyaki", "katsu", "tempura", "edamame", "bento", "takoyaki"],
    "Tacos": ["taco"],
    "Sushi" : ["sushi"],
    "Mediterranean" : ["mediterranean","pita", "damascus", "greek", "greece", "briam","taramasalata", "opa"],
    "Hot Dogs & Sausages" : ["hot dogs","sausage", "weiner"],
    "Middle Eastern" : ["middle eastern","falafel", "hummus","shawarma", "baklava", "donair", "tzatziki"],
    "Convenience" : ["convenience","pre-made","grocery", "slurpee"],
    "Mexican" : ["mexican", "chilaquiles", "burrito", "nacho", "quesadilla", "queso", "taquito", "salsa"],
    "Steakhouse" : ["steakhouse", "steak"],
    "Halal" : ["halal", "zabiha"],
    "Korean" : ["korean", "kimchi", "bulgogi","bibimbap","tteokbokki","jjambbong","doenjang"],
    "Thai" : ["thai", "tom yum goong", "green curry"],
    "Soup" : ["soup"],
    "Gluten Free" : ["gluten free", "no gluten"],
    "Popcorn": ["popcorn"], 
    "Pet Food" : ["pet", "dog", "cat"],
    "Bubble Tea" : ["bubble tea", "boba", "milk tea", "taro milk"],
    "French" : ["french", "francais", "crepe", "foie gras","coq au vin","cassoulet","baguette","croissant","gougeres","cajun & creole", "creole"],
    "African" : ["african","pap en vleis", "shisa nyama","bunny chow","koshari"],
    "Latin American" : ["latin","asado","saltena","feijoada","empanada","bandeja paisa",
                        "gallo pinto","ropa vieja","mangu","encebollado", "pupusas","pepian","peruvian"],
    "Haute Cuisine" : ["haute", "high class", "expensive", "champagne"],
    "Ethiopian" : ["ethiopian","tibs","kitfo","beyainatu","fuul"],
    "Caribbean" : ["caribbean", "jamaica","barbados","bahamas"],
    "Filipino" : ["filipino","adobo","lechon","sisig","bulalo"],
    "Spanish" : ["spanish","paella valenciana","patatas bravas","gazpacho","pimientos de padron","jamon","tapas","churro"],
    "Butcher" : ["raw", "butcher","delicatessen"],
    "Kosher" : ["kosher", 'kashrut', 'jewish'],
    "German" : ["german","schnitzel","rouladen","eintopf","sauerbraten"]
}
print(len(list(cuisine_map.keys())))

54


In [43]:
# hit = 0
# all = 0
# test = []
# test2 = []
# def f(x, y):
#   global hit, all
#   ret_val = False
#   res = isinstance(x, str)
#   if not res:
#     print(x)
#     return False
#   any_items_found = False
#   item_found = False
#   all +=1
#   temp_item = x.lower()
#   flag = False
#   for key in cuisine_map:
#     if flag: 
#       break
#     for food in cuisine_map[key]:
#       if food in temp_item:
#         hit += 1
#         ret_val = True
#         any_items_found = True
#         flag = True
#         item_found = True
#         break
#   if not item_found:
#     test.append(temp_item)
#     test2.append(y) 
#         # return True
#   # if not any_items_found:
#   #   r_name = y.lower()
#   #   flag = False
#   #   for key in cuisine_map:
#   #     if flag:
#   #       break
#   #     for food in cuisine_map[key]:
#   #       if food in r_name:
#   #         hit +=1 
#   #         flag = True
#   #         ret_val = True
#   #         break
#   return ret_val


# result = [f(x,y) for x,y in zip(rf['item description'],rf['short_name'])]
# print(sum(result))
# print(len(result))
# print(hit)
# print(all)
# print(hit/all*100)

In [44]:
overall_cuisine = dict(zip(cuisine_map.keys(), [0] * 54))
grouped = rf.groupby('short_name')
restaurant_profile_list = []
miss = []
nooblet = 0
def process_group(group):
    global miss
    global nooblet
    global overall_cuisine
    restaurant_profile = dict(zip(cuisine_map.keys(), [0] * 54))
    total_filtered = 0
    for index, row in group.iterrows():
        any_items_found = False
        lower_item = row['item_description'].lower()
        cuisine_identified_item = False
        for key in cuisine_map:
            for cuisine in cuisine_map[key]:
                if cuisine in lower_item:
                    if key in ethnic_or_cultural_items:
                        cuisine_identified_item = True  
                    restaurant_profile[key] +=1
                    overall_cuisine[key] += 1
                    total_filtered +=1 
                    any_items_found = True
                    break
        if not cuisine_identified_item:
            nooblet +=1
            restaraunt_cuisines_df = rf_c[rf_c['short_name'] == row['short_name']]['cussine_list']
            if restaraunt_cuisines_df.shape[0] > 0:
                restaraunt_cuisines_list = restaraunt_cuisines_df.str.split(", ")
                restaraunt_cuisines = restaraunt_cuisines_list.iloc[0]
                for cuisine in restaraunt_cuisines:
                    restaurant_profile[cuisine] +=1 
                    overall_cuisine[cuisine] += 1
                    total_filtered += 1 
        if not any_items_found:
            miss.append(lower_item)      
    map_total_filtered = 0
    for value in restaurant_profile.values():
          map_total_filtered += value
    if total_filtered == map_total_filtered and total_filtered != 0:
        for key in restaurant_profile:
            restaurant_profile[key] = restaurant_profile[key]/total_filtered
        restaurant_profile['short_name'] = group.name
        restaurant_profile_list.append(restaurant_profile)
    else:
      print("missed restauraunt: ", group.name)

grouped.apply(process_group)
print(len(miss))

47475


In [49]:
print(miss)

['haus hot sauce extra sides ', 'avocado extra sides add avocado slices on the side.', 'haus ranch sauce wbh haus sauces our in haus made ranch sauce.', 'number of plastic cutlery place settings how many people will be eating from this order?', 'market dip handhelds pulled short rib, coleslaw, crispy onions, monterey jack cheese, horseradish aioli, ciabatta, and au-jus.', 'honey hot sauce extra sauce sweet and spicy.', 'haus hot sauce wbh haus sauces our in haus made hot sauce. mild spice.', 'haus togarashi wbh haus sauces our haus made togarashi sauce.', 'wbh haus 3-set wbh haus sauces a three pack that includes our in haus made hot sauce, togarashi and ranch.', 'honey truffle aioli sauce extra sides ', 'sour puss raspberry (750 ml) liqueurs (15% abv)', 'crown royal (750 ml) whiskey (40% abv)', 'havana club 7 year old rum (750 ml) rum (40% abv)', 'tanqueray london dry gin (375 ml) mickeys (40% abv)', "michter's small batch bourbon (750 ml) bourbon (45.7% abv)", 'talisker scotch  port 

In [45]:
restaurant_profile_df = pd.DataFrame(restaurant_profile_list)
print(rf['short_name'].nunique())
print(restaurant_profile_df['short_name'].nunique())

2354
2354


In [46]:
restaurant_profile_df = restaurant_profile_df.reindex(columns=['short_name'] + list(restaurant_profile_df.columns.drop('short_name')))
cols = restaurant_profile_df.columns.tolist()
cols = cols[:-54] + sorted(cols[-54:])
restaurant_profile_df = restaurant_profile_df.reindex(columns=cols)
print(restaurant_profile_df)

                    short_name  African   Alcohol    Bakery  Barbecue  \
0         1600 world bier haus      0.0  0.007194  0.021583  0.028777   
1     17th ave liquor boutique      0.0  0.710243  0.008086  0.000674   
2                 1886 buffalo      0.0  0.000000  0.105556  0.066667   
3            1886 buffalo cafe      0.0  0.000000  0.011494  0.045977   
4        19 kitchen cooking nw      0.0  0.000000  0.000000  0.291667   
...                        ...      ...       ...       ...       ...   
2349               zen 8 grill      0.0  0.001996  0.033932  0.007984   
2350      zero latency on tilt      0.0  0.137931  0.034483  0.017241   
2351       zimidara restaurant      0.0  0.000000  0.000000  0.000000   
2352                     zushi      0.0  0.000000  0.043478  0.000000   
2353       zyn the wine market      0.0  0.828729  0.005525  0.000000   

          Beef  Breakfast & Brunch  Bubble Tea   Burgers   Butcher  ...  \
0     0.014388            0.050360         0.0  

In [47]:
# Make sure each row adds to 1 
for index, row in restaurant_profile_df.iterrows():
    if row[1:].sum() > 1.05 or row[1:].sum() < 0.95:
      print(row[1:].sum())
      print("Something is wrong at index: ", index)

In [48]:
print(restaurant_profile_df)
restaurant_profile_df.to_csv('./derived_files/restaurant_profiles.csv', index=False)

                    short_name  African   Alcohol    Bakery  Barbecue  \
0         1600 world bier haus      0.0  0.007194  0.021583  0.028777   
1     17th ave liquor boutique      0.0  0.710243  0.008086  0.000674   
2                 1886 buffalo      0.0  0.000000  0.105556  0.066667   
3            1886 buffalo cafe      0.0  0.000000  0.011494  0.045977   
4        19 kitchen cooking nw      0.0  0.000000  0.000000  0.291667   
...                        ...      ...       ...       ...       ...   
2349               zen 8 grill      0.0  0.001996  0.033932  0.007984   
2350      zero latency on tilt      0.0  0.137931  0.034483  0.017241   
2351       zimidara restaurant      0.0  0.000000  0.000000  0.000000   
2352                     zushi      0.0  0.000000  0.043478  0.000000   
2353       zyn the wine market      0.0  0.828729  0.005525  0.000000   

          Beef  Breakfast & Brunch  Bubble Tea   Burgers   Butcher  ...  \
0     0.014388            0.050360         0.0  